In [6]:
import os
import torch
import torch.optim
import numpy as np
import pandas as pd
from PIL import Image
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader

In [ ]:
!unzip cuts.zip

In [2]:
PATH_new = '/content/cuts'
Category = {0: 'ROTE', 1: 'SANE'}

In [3]:
data_transforms = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(), # data augmentation
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [12]:
# read csv file to DataFrame
terns_columns = ['filename', 'xmin', 'ymin', 'xmax', 'ymax']
terns_to_label = pd.read_csv('/content/cuts/beach_tern_labels.csv', header=0, names=terns_columns)
terns_to_label['filename'] = terns_to_label['filename'].astype(str) + '.jpg'

test_terns = []
for i in range(terns_to_label.shape[0]):
  img_name = os.path.join(PATH_new, terns_to_label.iloc[i, 0])
  image = Image.open(img_name).convert('RGB')
  image = data_transforms(image)
  test_terns.append(image)

test_terns = torch.stack(test_terns)
test_terns.shape

torch.Size([617, 3, 224, 224])

In [8]:
device = torch.device('cuda')
testModel = models.resnet18()
# testModel.to(device)
in_features = testModel.fc.in_features
testModel.fc = torch.nn.Linear(in_features, 2)

In [9]:
testModel.load_state_dict(torch.load('/content/model_weights.pth'))

<All keys matched successfully>

In [10]:
def predict(model, test_terns):
  model.eval()   
  predictions = []

  with torch.no_grad():
    # Iterate over data.
      outputs = model(test_terns).detach().cpu().numpy()
      preds = outputs.argmax(axis=1)
      predictions.extend(preds)

  return np.array(predictions)

In [11]:
preds = (predict(testModel, test_terns))

In [13]:
preds

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [14]:
# mapping the true label to DataFrame
labelsPred = []
for pred in preds:
    labelsPred.append(Category[pred])

In [15]:
terns_to_label["predictLabel"] = np.array(labelsPred)

In [16]:
terns_to_label.to_csv("cutsPred.csv", header=True)